In [2]:
import os
import re
import pandas as pd
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import logging
from tqdm import tqdm

In [ ]:
def parse_hotel_reviews(data_dir):
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    # Updated regex pattern to handle flexible spacing and separators
    date_pattern = re.compile(r'(\b(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{1,2}\s\d{4})\s+([^\t\n]*)\s+(.*)')
    data_frames = []

    for root, dirs, files in os.walk(data_dir):
        city = os.path.basename(root)
        for file in files:
            file_path = os.path.join(root, file)
            hotel_name = os.path.splitext(file)[0]  # Assuming the file has no extension
            try:
                with open(file_path, 'r', encoding='ISO-8859-1') as f:
                    content = f.read()

                if not content:
                    logging.warning(f'File {file_path} is empty.')
                    continue

                reviews = date_pattern.findall(content)
                if not reviews:
                    logging.warning(f'No reviews found in file {file_path}. Content: {content[:500]}')
                    continue

                reviews_data = [{'hotel_name': hotel_name, 'hotel_city': city, 'review_date': date, 'hotel_review': review.strip()} for date, _, title, review in reviews]
                if reviews_data:
                    data_frames.append(pd.DataFrame(reviews_data))

            except Exception as e:
                logging.error(f'Error processing file {file_path}: {e}')
                continue

    if data_frames:
        review_df = pd.concat(data_frames, ignore_index=True)
    else:
        review_df = pd.DataFrame(columns=['hotel_name', 'hotel_city', 'review_date', 'hotel_review'])

    return review_df

data_dir = 'city_data'
review_df = parse_hotel_reviews(data_dir)
print(review_df.head())
review_df.to_csv('csv_city/derlenmis_reviews.csv', index=False)

                                          hotel_name hotel_city  review_date  \
0  uk_england_london_130_queensgate_london_apartm...  city_data  Aug 14 2007   
1  uk_england_london_130_queensgate_london_apartm...  city_data  Oct 17 2006   
2  uk_england_london_130_queensgate_london_apartm...  city_data   Aug 9 2006   
3  uk_england_london_130_queensgate_london_apartm...  city_data   Aug 8 2006   
4  uk_england_london_130_queensgate_london_apartm...  city_data  Jun 12 2006   

                                        hotel_review  
0  I've stayed at the Queensgate on several occas...  
1  We stayed at 130 Queensgate Apartments for 6 d...  
2  Having read the reviews on this site, I though...  
3  This is a foul hotel. From the dingy, peeling ...  
4  130 Queensgate is, as others state, a bit drea...  


In [ ]:
review_df = pd.read_csv('csv_city/derlenmis_reviews.csv')
review_df = review_df.dropna(subset=['hotel_review'])
review_df['processed_review'] = review_df['hotel_review'].str.lower().str.replace('\t', ' ', regex=False)
review_df.to_csv('csv_city/processed_reviews.csv', index=False)

In [ ]:
review_df = pd.read_csv('csv_city/processed_reviews.csv')
review_df = review_df.drop('hotel_review', axis=1)
review_df = review_df.drop('review_date', axis=1)
review_df.to_csv('csv_city/processed_reviews2.csv', index=False)

In [10]:
import torch

absa_tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
absa_model = AutoModelForSequenceClassification.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")

c:\Users\durud\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\durud\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [4]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

aspects = ['cleanliness', 'room', 'service', 'location', 'value', 'safety', 'comfort', 'transportation', 'noise']
weights = {'negative': -1, 'neutral': 0, 'positive': 1}

aspect_keywords = {
    'cleanliness': ['clean', 'dirty', 'smell', 'stink', 'stunk', 'filthy'],
    'room': ['room', 'bed', 'suite', 'large'],
    'service': ['service', 'staff', 'help', 'support'],
    'location': ['location', 'close', 'area', 'far'],
    'value': ['value', 'worth', 'price'],
    'safety': ['safe', 'safety', 'secure', 'danger', 'dangerous'],
    'comfort': ['comfort', 'comfortable', 'uncomfortable'],
    'transportation': ['bus', 'metro', 'station', 'close', 'walk'],
    'noise': ['sound', 'volume', 'noisy', 'noise']
}

In [5]:
batch_size = 16

def process_batch(batch_reviews, aspect):
    aspect_reviews = [f"{aspect}: {review}" for review in batch_reviews]
    inputs = absa_tokenizer(aspect_reviews, return_tensors="pt", padding=True, truncation=True)
    outputs = absa_model(**inputs)
    probs = F.softmax(outputs.logits, dim=-1)
    return probs.detach().numpy()

def is_aspect_mentioned(review, aspect):
    keywords = aspect_keywords[aspect]
    return any(keyword in review for keyword in keywords)


In [7]:

review_df = pd.read_csv('csv3/processed_reviews2.csv')
# Adding columns for overall sentiment scores
for aspect in aspects:
    review_df[f'{aspect}_score'] = 0

In [8]:

for aspect in aspects:
    logging.info(f"Starting processing for aspect: {aspect}")
    overall_scores = []
    for i in tqdm(range(0, len(review_df), batch_size), desc=f"Batches for {aspect}"):
        batch_reviews = review_df['processed_review'][i:i + batch_size]
        aspect_mentioned = [is_aspect_mentioned(review, aspect) for review in batch_reviews]
        if any(aspect_mentioned):
            probs = process_batch([review for review, mentioned in zip(batch_reviews, aspect_mentioned) if mentioned], aspect)
            scores = [sum(weights[sentiment] * probs[j, k] for k, sentiment in enumerate(['negative', 'neutral', 'positive'])) for j in range(len(probs))]
        else:
            scores = [0] * len(batch_reviews)
        for idx, score in zip(batch_reviews.index, scores):
            review_df.at[idx, f'{aspect}_score'] = score

    logging.info(f"Finished processing for aspect: {aspect}")

# Save the final DataFrame with overall scores
review_df.to_csv('deneme/overall_sentiment_scores.csv', index=False)

2024-05-15 16:21:05,625 - INFO - Starting processing for aspect: cleanliness
Batches for cleanliness:   0%|          | 0/6 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
C:\Users\durud\AppData\Local\Temp\ipykernel_18836\3124420602.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9822808085009456' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  review_df.at[idx, f'{aspect}_score'] = score
Batches for cleanliness: 100%|██████████| 6/6 [00:38<00:00,  6.50s/it]
2024-05-15 16:21:44,626 - INFO - Finished processing for aspect: cleanliness
2024-05-15 16:21:44,628 - INFO - Starting processing for aspect: room
Batches for room:   0%|          | 0/6 [00:00<?, ?it/s]C:\Users\durud\AppData\Local\Temp\ipykernel_18836\3124420602.py:13: FutureWarning: Setting an it